In [1]:
import os
import re
import sys
import pandas as pd
import glob
import numpy as np

import itertools
from itertools import tee


In [4]:
virus_coord_dir = "/Users/u241374/mike_tisza/sandbox/ct2_b_caccae_f/ct2_tmp/assess_prune/indiv_seqs"

gene_annotation_file = "/Users/u241374/mike_tisza/sandbox/ct2_b_caccae_f/ct2_tmp/assess_prune/contig_gene_annotation_summary.tsv"

oriented_contigs_file = "/Users/u241374/mike_tisza/sandbox/ct2_b_caccae_f/ct2_tmp/oriented_hallmark_contigs.fasta"

In [8]:
try:

    coord_files = glob.glob(os.path.join(virus_coord_dir, "*.viruses.tsv"))

    df_from_each_coord = (pd.read_csv(coord, sep = "\t", header = None,
                                    names = ["contig", "chunk_start", "chunk_stop", 
                                             "chunk_name", "hallmark_count"])
                                    for coord in coord_files)
    chunk_coord_df = pd.concat(df_from_each_coord, ignore_index=True)

    #chunk_coord_df.head()

except:
    print("no chunk coords")

chunk_coord_df.head()

,contig,chunk_start,chunk_stop,chunk_name,hallmark_count
0,ct2_b_caccae_f1,1273382,1347154,Chunk_83,5
1,ct2_b_caccae_f1,1725318,1742123,Chunk_117,2
2,ct2_b_caccae_f1,2076488,2116229,Chunk_147,8
3,ct2_b_caccae_f1,4213532,4335480,Chunk_289,8
